In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns

In [3]:
sns.set_context('poster', font_scale=1.25)

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from LB_D2Q9 import pipe_cython as lb
from LB_D2Q9 import pipe_opencl as lb_cl

We now want to visualize the evolution of the flow...let's see how it goes...

In [6]:
from IPython import display

It's probably easier to specify dt and dr and the dimensions. Then get everything else from that...yeah. That way the velocity is on the scale of U ~ 0.1 or so.

In [7]:
#### Input to the simulation in SI. ######
diameter = 2. # meters
length = 4. # meters

deltaP = -.1

dr = 0.005
dt = dr**2

viscosity = 10.**-6. # More viscous = easier to simulate!

##### Derived parameters ######
print 'Desired viscosity:' , viscosity
#Re = (input_velocity * diameter)/viscosity
#print 'Desired Re', Re

# Re is set by diameter, input_velocity, and viscosity
# Adjust dr so that mach number doers not explode!
print
print '###### Simulation Parameters #########'
print

# Solve for the desired omega...make sure it's in the correct range.


ly = int(np.ceil(diameter/dr))
lx = int(np.ceil(length/dr))
print 'ly', ly
print 'lx', lx

nx = lx + 1
ny = ly + 1

omega = .5 + (3*dt*viscosity)/(dr**2)

print 'omega:' , omega

assert (omega > 0.5) and (omega < 1)

obstacle_size=.25 # meters

Desired viscosity: 1e-06

###### Simulation Parameters #########

ly 400
lx 800
omega: 0.500003


In [8]:
nx*ny*9

2890809

In [9]:
import skimage as ski
import skimage.draw

In [10]:
import sys

In [11]:
sim = lb.Pipe_Flow(lx=lx, ly=ly, dr=dr, dt=dt, omega=omega,
                  deltaP = deltaP)

In [13]:
two_d_local_size = (128, 32)
three_d_local_size = (32, 32, 3)

sim_cl = lb_cl.Pipe_Flow(lx=lx, ly=ly, dr=dr, dt=dt, omega=omega,
                  deltaP = deltaP, two_d_local_size=two_d_local_size,
                        three_d_local_size=three_d_local_size)

The platforms detected are:
---------------------------
Apple Apple version: OpenCL 1.2 (Sep 21 2015 19:24:11)
The devices detected on platform Apple are:
---------------------------
Intel(R) Core(TM) i7-5557U CPU @ 3.10GHz [Type: CPU ]
Maximum clock Frequency: 3100 MHz
Maximum allocable memory size: 4294 MB
Maximum work group size 1024
Maximum work item dimensions 3
Maximum work item size [1024, 1, 1]
---------------------------
Intel(R) Iris(TM) Graphics 6100 [Type: GPU ]
Maximum clock Frequency: 1100 MHz
Maximum allocable memory size: 402 MB
Maximum work group size 256
Maximum work item dimensions 3
Maximum work item size [256, 256, 256]
---------------------------
This context is associated with  2 devices
2d global: (896, 416)
2d local: (128, 32)
3d global: (832, 416, 9)
3d local: (32, 32, 3)


//anaconda/lib/python2.7/site-packages/pyopencl-2015.1-py2.7-macosx-10.5-x86_64.egg/pyopencl/__init__.py:59: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) Iris(TM) Graphics 6100' on 'Apple' at 0x1024500> succeeded, but said:

<program source>:17:29: warning: double precision constant requires cl_khr_fp64, casting to single precision
        const float w[9] = {4./9.,1./9.,1./9.,1./9.,1./9.,1./36., 1./36.,1./36.,1./36.}; // weights for directions
                            ^
<program source>:17:32: warning: double precision constant requires cl_khr_fp64, casting to single precision
        const float w[9] = {4./9.,1./9.,1./9.,1./9.,1./9.,1./36., 1./36.,1./36.,1./36.}; // weights for directions
                               ^
<program source>:17:35: warning: double precision constant requires cl_khr_fp64, casting to single precision
        const float w[9] = {4./9.,1./9.,1./9.,1./9.,1./9.,1./36., 1./36

LogicError: clEnqueueNDRangeKernel failed: invalid work group size

In [ ]:
%%prun

sim.run(100)

In [ ]:
%%prun

sim_cl.run(100)

Let's make sure the velocity on the walls is basically zero...

In [ ]:
fields = sim_cl.get_fields_on_cpu()
u = fields['u']
v = fields['v']
mag_cl = np.sqrt(u**2 + v**2)
plt.imshow(mag_cl.T, cmap=cm.coolwarm)
plt.grid(False)
#plt.clim(0, 1)
plt.colorbar()

In [ ]:
mag = np.sqrt(sim.u**2 + sim.v**2)
plt.imshow(mag.T, cmap=cm.coolwarm)
plt.grid(False)
#plt.clim(0,1)
plt.colorbar()

Well, this basically looks correct...let's compare the two. The corners are definitely different though. And the walls.

In [ ]:
differences = np.abs(mag_cl - mag) > 10.**-6.
plt.imshow(differences.T, interpolation=None)
plt.colorbar()
plt.grid(False)

In [ ]:
plt.imshow((mag_cl - mag).T)
plt.grid(False)
plt.colorbar()

Interesting. So the problem areas appear to be the walls now...and only the walls! Interesting...

Well...it looks reasonably correct now...

In [ ]:
plt.imshow((mag_cl - mag).T)
plt.grid(False)
plt.colorbar()

OMG. I think it works! :D

In [ ]:
def check_variable(varname):
    sim_cl_field = sim_cl.get_fields_on_cpu()[varname]
    
    if varname == 'rho':
        sim_field = sim.rho
    if varname == 'u':
        sim_field = sim.u
    if varname == 'v':
        sim_field = sim.v
    if varname == 'feq':
        sim_field = sim.feq
        # Have to reshape it...ugh.
        sim_field = np.rollaxis(sim_field, 0, 3)
    if varname == 'f':
        sim_field = sim.f
        sim_field = np.rollaxis(sim_field, 0, 3)
    return sim_field, sim_cl_field

In [ ]:
sim.move_bcs()
sim_cl.move_bcs()

In [ ]:
sim.move()
sim_cl.move()

I'm so confused. Somehow, running the move_bcs command did nothing...

In [ ]:
field, field_cl = check_variable('f')

In [ ]:
# They should be exactly the same
fieldnum = 1
problems = np.abs(field[:, :, fieldnum] - field_cl[:, :, fieldnum]) > 10.**-6.
plt.imshow(problems.T)
plt.clim(0, 1)
plt.colorbar()
plt.grid(False)

Ok, interesting...there must be something wrong with my boundaries for the different jumpers!

Ok, fields 6 & 8 are still not cooperating...we'll have to figure out why that is.

Now, everything is fine...lol

In [ ]:
print field[5, 5, 3]
print field_cl[5, 5, 3]

Interesting...why does the opencl seem to be better behaved? lol

6 and 8 are the problems. evidently.

In [ ]:
problems[lx, ly]

Only the edges appear crazy...

In [ ]:
rho = sim.get_fields_on_cpu()['rho']
print np.max(rho)
print np.min(rho)

In [ ]:
fields = sim.get_fields_on_cpu()
plt.imshow(fields['rho'].T)
plt.colorbar()
plt.grid(False)

In [ ]:
fields = sim.get_fields_on_cpu()
plt.imshow(fields['u'].T)
plt.colorbar()
plt.grid(False)

Yeah the boundaries still look utterly psychotic. lol. Not sure what's up with them! F & feq still look reasonable...so it's probably in an update function...

In [ ]:
fields = sim.get_fields_on_cpu()
plt.imshow(fields['v'].T)
plt.colorbar()
plt.grid(False)

In [ ]:
fields = sim_cl.get_fields_on_cpu()
plt.imshow(np.sum(fields['feq'],axis=2).T)
plt.colorbar()
plt.grid(False)

In [ ]:
fields = sim.get_fields_on_cpu()
plt.imshow(np.sum(fields['f'], axis=2).T)
plt.colorbar()
plt.grid(False)

In [ ]:
sim.run(10)

In [ ]:
wat = sim.get_fields_on_cpu()['u']**2 + sim.get_fields_on_cpu()['v']**2
plt.imshow(wat.T, cmap = cm.coolwarm)
plt.grid(False)
plt.colorbar()

Yeah this is not correct lol. It explodes!

In [ ]:
sim.get_fields_on_cpu()['rho']

LOL

Interesting...weird streaks are forming! Disaster...lol.

In [ ]:
total_num_steps = 5000
steps_per_image= 25

num_to_record = int(np.ceil(float(total_num_steps)/steps_per_image))

Re_list = []
Ma_list = []

plt.figure()
for i in range(num_to_record):
    sim.update_dimensionless_nums()
    Re_list.append(sim.Re)
    Ma_list.append(sim.Ma)
    print 'max non_dim u:', np.max(sim.u)
    print 'sum of mass' , np.sum(sim.rho)/(sim.nx*sim.ny)
    mag = np.sqrt(sim.u**2 + sim.v**2)
    plt.imshow(mag.T, cmap = cm.coolwarm)
    plt.grid(False)
    #plt.clim(vmin=0, vmax=1.)
    #if i == 0:
    #    plt.colorbar()
    display.clear_output(wait=True)
    display.display(plt.gcf())
    
    sim.run(steps_per_image)
plt.clf() # This way a second figure does not appear at the end...

IDK how to convert...but let's just fit.

In [ ]:
plt.semilogy(Ma_list, ls='-', marker='.')

Nope, it's fucked! lol.

In [ ]:
plt.semilogy(Re_list, ls='-', marker='.')

Looks like it still hasn't converged. But, as expected, larger pressure differences leads to larger reynolds numbers!